# Introduction to RAMP platform and interaction with scikit-learn

RAMP is a Kaggle-like platform. It is used to run data science challenge. Indeed, a challenge is organized around a specific problem for which the data and the evaluation are already defined. Participants will only have to focus on the development of the machine learning pipeline.

Here, we will present how the RAMP platform works. RAMP relies on a data science problem which is formulated in `problem.py`. It defines both data and evaluation. If you are interested, you can open this file. Otherwise, we will only used a couple of the function defined there.

In [1]:
import problem

We will load the training and testing datasets available for the challenge.

In [2]:
X_train, y_train = problem.get_train_data()

In [3]:
X_test, y_test = problem.get_test_data()

Succently, we can check the type of features in `X` and the target that we would like to predict `y`.

In [4]:
X_train.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [5]:
y_train[:5]

array([12.33129622, 10.77518151, 11.08317675, 11.16926784, 11.26936373])

The target `y` corresponds to a number of passengers (modified using a `log` function). Associated with each target, we have an information in `X` related to the date (`DateOfDeparture`) and the airports of departure (`Departure`) and arrival (`Arrival`). Besides, we have the information regarding the mean (`WeeksToDeparture`) and standard deviation (`std_wtd`) of the time in weeks between the booking and the departure.

So we try to answer to the following information: **With some flying information between airports, can we predict the (log) flow of passengers?**

Let's make a basic scikit-learn model that could use some the data in `X` to answer this question. We will create a factory function `get_estimator()` to return the model.

In [6]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

def get_estimator():
    cat_processor = OrdinalEncoder()
    cat_columns = ["Departure", "Arrival"]

    num_processor = "passthrough"
    num_columns = ["WeeksToDeparture", "std_wtd"]

    preprocessor = make_column_transformer(
        (cat_processor, cat_columns),
        (num_processor, num_columns),
        remainder="drop",  # drop the unused columns
    )

    return make_pipeline(preprocessor, RandomForestRegressor())

At this stage, we could train and test our model using scikit-learn

In [7]:
import numpy as np
from sklearn.model_selection import cross_val_score

model = get_estimator()
cv = problem.get_cv(X_train, y_train)

scores = cross_val_score(
    model, X_train, y_train, cv=cv,
    scoring="neg_mean_squared_error",
)
rmse_scores = np.sqrt(-scores)
for cv_idx, score in enumerate(rmse_scores):
    print(f"CV Fold #{cv_idx}: {score:.3f}")
print(
    f"RMSE = {rmse_scores.mean():.3f} "
    f"+/- {rmse_scores.std():.3f}"
)

CV Fold #0: 0.854
CV Fold #1: 0.862
CV Fold #2: 0.851
CV Fold #3: 0.875
CV Fold #4: 0.864
CV Fold #5: 0.850
CV Fold #6: 0.846
CV Fold #7: 0.839
RMSE = 0.855 +/- 0.011


RAMP was developed to avoid running the last cell. Instead, the idea is to store the content of cell where `get_estimator` is defined into a file. For this challenge, the name of the file is called `estimator.py`. You can check the content of `submissions/starting_kit` directory.

Let's make our first kit. Alongside of the three kits, create a folder `my_first_kit` in the folder `submissions`. The command below will create a `estimator.py` file into this directory.

In [8]:
%%writefile submissions/my_first_kit/estimator.py

from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

def get_estimator():
    cat_processor = OrdinalEncoder()
    cat_columns = ["Departure", "Arrival"]

    num_processor = "passthrough"
    num_columns = ["WeeksToDeparture", "std_wtd"]

    preprocessor = make_column_transformer(
        (cat_processor, cat_columns),
        (num_processor, num_columns),
        remainder="drop",  # drop the unused columns
    )

    return make_pipeline(preprocessor, RandomForestRegressor())

Overwriting submissions/my_first_kit/estimator.py


Once the `estimator.py` was created, we can use the `ramp-test` command to automatically execute the evaluation on our newly created estimator. 

In [9]:
!ramp-test --submission my_first_kit

Testing Number of air passengers prediction
Reading train and test files from ./data ...
Reading cv ...
Training submissions/my_first_kit ...
CV fold 0
	score   rmse      time
	train  0.323  1.898484
	valid  0.854  0.174880
	test   0.886  0.055788
CV fold 1
	score   rmse      time
	train  0.316  1.230795
	valid  0.863  0.164316
	test   0.879  0.054035
CV fold 2
	score   rmse      time
	train  0.321  1.292139
	valid  0.847  0.234369
	test   0.875  0.058681
CV fold 3
	score   rmse      time
	train  0.307  1.593192
	valid  0.875  0.232582
	test   0.887  0.073297
CV fold 4
	score   rmse      time
	train  0.317  1.648690
	valid  0.861  0.293554
	test   0.901  0.088616
CV fold 5
	score   rmse      time
	train  0.319  1.798566
	valid  0.849  0.268945
	test   0.889  0.085102
CV fold 6
	score   rmse      time
	train  0.313  2.117580
	valid  0.844  0.355601
	test   0.882  0.110186
CV fold 7
	score   rmse      time
	train  0.319  1.936658
	valid  0.839  0.252511
	test   0.890  0.093794
----------

This command is the exact replica of what would happen on our server when you will submit your file `estimator.py` on https://ramp.studio.

We can now make a demo on submitting this kit on the RAMP platform.